In [1]:
import pandas as pd
import numpy as np
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import shap

# Imports pour la modélisation et l'évaluation
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score, recall_score
import lightgbm as lgb

# Imports pour le tracking et l'optimisation
import mlflow
import optuna
from optuna.integration.mlflow import MLflowCallback

warnings.filterwarnings('ignore')

In [11]:
# --- SECTION 1 : FONCTIONS ---

def calculate_metrics_and_optimal_threshold(y_true, y_pred_proba, cost_fn=10, cost_fp=1):
    """
    Calcule un ensemble de métriques et trouve le meilleur seuil pour minimiser un coût métier.

    Args:
        y_true (pd.Series): Les vraies étiquettes (0 ou 1).
        y_pred_proba (np.array): Les probabilités prédites pour la classe 1.
        cost_fn (int): Coût d'un Faux Négatif.
        cost_fp (int): Coût d'un Faux Positif.

    Returns:
        dict: Un dictionnaire contenant l'AUC, le coût minimum, le meilleur seuil,
              et les scores F1 et Recall à ce seuil.
    """
    # Calcul de l'AUC
    oof_auc = roc_auc_score(y_true, y_pred_proba)

    # Recherche du meilleur seuil pour minimiser le coût métier
    thresholds = np.linspace(0.01, 0.99, 100)
    costs, f1_scores, recall_scores = [], [], []

    for thresh in thresholds:
        preds_binary = (y_pred_proba > thresh).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, preds_binary).ravel()
        
        costs.append((fp * cost_fp) + (fn * cost_fn))
        f1_scores.append(f1_score(y_true, preds_binary))
        recall_scores.append(recall_score(y_true, preds_binary, pos_label=1))

    # Trouver l'index du coût minimum
    best_idx = np.argmin(costs)
    
    # Récupérer les métriques à cet index optimal
    metrics = {
        'oof_auc': oof_auc,
        'min_business_cost': costs[best_idx],
        'best_threshold': thresholds[best_idx],
        'f1_at_best_threshold': f1_scores[best_idx],
        'recall_at_best_threshold': recall_scores[best_idx],
    }
    return metrics

def train_and_evaluate_model(model_name, model_pipeline, X, y):
    """
    Entraîne un modèle en validation croisée, calcule les métriques et loggue tout dans MLflow.

    Args:
        model_name (str): Le nom à donner à la run MLflow.
        model_pipeline: Le modèle ou pipeline scikit-learn à entraîner.
        X (pd.DataFrame): Les features.
        y (pd.Series): La variable cible.

    Returns:
        float: Le coût métier minimum pour ce modèle.
    """
    with mlflow.start_run(run_name=model_name):
        print(f"\n--- Démarrage de la run : {model_name} ---")
        mlflow.autolog(disable=True)
        mlflow.log_param("model_name", model_name)

        # Entraînement en validation croisée
        N_SPLITS = 5
        skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
        oof_preds = np.zeros(len(X))

        for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            print(f"  Fold {fold + 1}/{N_SPLITS}...")
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_val = X.iloc[val_idx]
            
            model_pipeline.fit(X_train, y_train)
            oof_preds[val_idx] = model_pipeline.predict_proba(X_val)[:, 1]

        # Calcul des métriques
        all_metrics = calculate_metrics_and_optimal_threshold(y, oof_preds)
        
        # Logging dans MLflow
        mlflow.log_metrics(all_metrics)
        print(f"  Résultats pour {model_name}:")
        for key, value in all_metrics.items():
            print(f"    {key}: {value:.4f}")
            
        # Création et sauvegarde du graphique du coût métier
        best_threshold_val = all_metrics['best_threshold']
        
        # Recalculer les coûts pour le graphique
        thresholds = np.linspace(0.01, 0.99, 100)
        costs = []
        cost_fn = 10
        cost_fp = 1
        for thresh in thresholds:
            preds_binary = (oof_preds > thresh).astype(int)
            tn, fp, fn, tp = confusion_matrix(y, preds_binary).ravel()
            costs.append((fp * cost_fp) + (fn * cost_fn))
        
        plt.figure(figsize=(10, 6))
        plt.plot(thresholds, costs)
        plt.vlines(best_threshold_val, ymin=min(costs), ymax=max(costs), colors='r', linestyles='--', label=f'Meilleur Seuil ({best_threshold_val:.2f})')
        plt.title(f'Coût métier vs Seuil pour {model_name}')
        plt.xlabel('Seuil de classification')
        plt.ylabel('Coût Métier Total')
        plt.legend()
        plt.grid(True)
        
        # Sauvegarder le graphique et le logger comme artefact
        plot_filename = f"{model_name}_cost_threshold.png"
        plt.savefig(plot_filename)
        mlflow.log_artifact(plot_filename)
        plt.close() # Fermer la figure pour ne pas l'afficher dans le notebook
        print(f"  Graphique du coût métier sauvegardé et loggué comme artefact : {plot_filename}")

        # Sauvegarde du modèle
        if "LogisticRegression" in model_name or "Dummy" in model_name:
            mlflow.sklearn.log_model(model_pipeline, f"{model_name}-pipeline")
        elif "LightGBM" in model_name:
             mlflow.lightgbm.log_model(model_pipeline, f"{model_name}-model")

        print(f"--- Fin de la run : {model_name} ---")
        return all_metrics['min_business_cost']
        

In [3]:
# --- SECTION 2 : CHARGEMENT ET PRÉPARATION DES DONNÉES ---

print("--- Chargement et Préparation des Données ---")
train_df = pd.read_csv('./data/application_train_rdy.csv')
print("Chargement des données d'entraînement réussi.")

# Remplacer les valeurs infinies par NaN
train_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Conversion des colonnes 'object' en numérique
cols_to_check = [col for col in train_df.columns if col not in ['TARGET', 'SK_ID_CURR']]
object_cols = train_df[cols_to_check].select_dtypes(include='object').columns
if len(object_cols) > 0:
    for col in object_cols:
        train_df[col] = pd.to_numeric(train_df[col], errors='coerce')

# Préparation finale de X et y
TARGET = 'TARGET'
y = train_df[TARGET]
X = train_df.drop(columns=[TARGET, 'SK_ID_CURR'])
X.columns = [re.sub(r'[^A-Za-z0-9_]+', '_', col) for col in X.columns]
print(f"Données prêtes avec {X.shape[1]} features.")


--- Chargement et Préparation des Données ---
Chargement des données d'entraînement réussi.
Données prêtes avec 796 features.


In [6]:
# --- SECTION 3 : EXPÉRIMENTATIONS DES MODÈLES DE BASE ---
EXPERIMENT_NAME = "Credit Scoring - Comparaison Modeles"
mlflow.set_experiment(EXPERIMENT_NAME)

model_scores = {}


In [12]:
# --- Modèle 1 : Dummy Classifier ---
dummy_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('classifier', DummyClassifier(strategy='stratified', random_state=42))
])
model_scores['Dummy_Classifier'] = train_and_evaluate_model("Dummy_Classifier_Baseline", dummy_pipeline, X, y)



--- Démarrage de la run : Dummy_Classifier_Baseline ---
  Fold 1/5...
  Fold 2/5...
  Fold 3/5...
  Fold 4/5...
  Fold 5/5...
  Résultats pour Dummy_Classifier_Baseline:
    oof_auc: 0.5000
    min_business_cost: 250986.0000
    best_threshold: 0.0100
    f1_at_best_threshold: 0.0803
    recall_at_best_threshold: 0.0799
  Graphique du coût métier sauvegardé et loggué comme artefact : Dummy_Classifier_Baseline_cost_threshold.png


2025/06/17 12:14:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


--- Fin de la run : Dummy_Classifier_Baseline ---


In [13]:
# --- Modèle 2 : Régression Logistique ---
logreg_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(class_weight='balanced', random_state=42, solver='liblinear'))
])
model_scores['Logistic_Regression'] = train_and_evaluate_model("Logistic_Regression_Baseline", logreg_pipeline, X, y)



--- Démarrage de la run : Logistic_Regression_Baseline ---
  Fold 1/5...
  Fold 2/5...
  Fold 3/5...
  Fold 4/5...
  Fold 5/5...
  Résultats pour Logistic_Regression_Baseline:
    oof_auc: 0.7727
    min_business_cost: 156040.0000
    best_threshold: 0.5247
    f1_at_best_threshold: 0.2882
    recall_at_best_threshold: 0.6685
  Graphique du coût métier sauvegardé et loggué comme artefact : Logistic_Regression_Baseline_cost_threshold.png
--- Fin de la run : Logistic_Regression_Baseline ---


In [14]:
# --- Modèle 3 : LightGBM ---
# Pour LightGBM, on impute les NaN mais on ne scale pas les données.
lgbm_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('classifier', lgb.LGBMClassifier(random_state=42))
])
model_scores['LightGBM'] = train_and_evaluate_model("LightGBM_Baseline", lgbm_pipeline, X, y)



--- Démarrage de la run : LightGBM_Baseline ---
  Fold 1/5...
[LightGBM] [Info] Number of positive: 19860, number of negative: 226145
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.408473 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 98871
[LightGBM] [Info] Number of data points in the train set: 246005, number of used features: 728
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080730 -> initscore=-2.432469
[LightGBM] [Info] Start training from score -2.432469
  Fold 2/5...
[LightGBM] [Info] Number of positive: 19860, number of negative: 226145
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.755328 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 99035
[LightGBM] [Info] Number of data points in the train set: 246005, number of used features: 730
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080730 -> i

2025/06/17 14:42:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


--- Fin de la run : LightGBM_Baseline ---


In [15]:
# --- SECTION 4 : SÉLECTION ET OPTIMISATION DU MEILLEUR MODÈLE ---

# Sélection du meilleur modèle basé sur le coût métier le plus bas
best_model_name = min(model_scores, key=model_scores.get)
print(f"\n--- Meilleur modèle de base sélectionné : {best_model_name} (Coût: {model_scores[best_model_name]}) ---")



--- Meilleur modèle de base sélectionné : LightGBM (Coût: 153218) ---


In [16]:
if "LightGBM" in best_model_name:
    print("\n--- Lancement de l'optimisation des hyperparamètres pour LightGBM avec Optuna ---")
    
    # MODIFIÉ : la fonction objective gère maintenant le logging MLflow
    def lgbm_objective(trial):
        with mlflow.start_run(run_name=f"LightGBM_optuna_trial_{trial.number}", nested=True):
            params = {
                'objective': 'binary', 'metric': 'auc', 'verbose': -1, 'n_jobs': -1, 'seed': 42,
                'n_estimators': 2000,
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, 300),
                'max_depth': trial.suggest_int('max_depth', 3, 12),
                'min_child_samples': trial.suggest_int('min_child_samples', 20, 200),
                'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            }
            mlflow.log_params(params)

            model = lgb.LGBMClassifier(**params)
            pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')), ('classifier', model)])

            N_SPLITS = 5
            skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
            oof_preds = np.zeros(len(X))
            for train_idx, val_idx in skf.split(X, y):
                X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
                X_val = X.iloc[val_idx]
                callbacks = [lgb.early_stopping(100, verbose=False)]
                pipeline.fit(X_train, y_train, classifier__callbacks=callbacks, classifier__eval_set=[(X_val, y.iloc[val_idx])])
                oof_preds[val_idx] = pipeline.predict_proba(X_val)[:, 1]
            
            metrics = calculate_metrics_and_optimal_threshold(y, oof_preds)
            mlflow.log_metrics(metrics)
            
            return metrics['min_business_cost']

    # On crée une run parente pour englober toute l'étude d'optimisation
    with mlflow.start_run(run_name="LightGBM_Optuna_Parent_Run"):
        study = optuna.create_study(direction="minimize")
        # MODIFIÉ : On enlève le callback, le logging est maintenant manuel
        study.optimize(lgbm_objective, n_trials=50)

        # Logging des meilleurs résultats dans la run parente
        print("\nOptimisation terminée ! Logging des meilleurs résultats...")
        best_trial = study.best_trial
        mlflow.log_params(best_trial.params)
        mlflow.log_metric("best_business_cost_overall", best_trial.value)
        print("Run de résumé des meilleurs résultats créée avec succès dans la run parente.")

print("\n--- Processus terminé ---")


[I 2025-06-17 15:55:24,740] A new study created in memory with name: no-name-4dc581e1-7cb5-460d-8f9f-3753a9433de2



--- Lancement de l'optimisation des hyperparamètres pour LightGBM avec Optuna ---


[I 2025-06-17 16:05:31,256] Trial 0 finished with value: 152246.0 and parameters: {'learning_rate': 0.012645836252328598, 'num_leaves': 97, 'max_depth': 12, 'min_child_samples': 133, 'subsample': 0.7934636795299609, 'colsample_bytree': 0.8559154146745591}. Best is trial 0 with value: 152246.0.
[I 2025-06-17 16:10:24,909] Trial 1 finished with value: 153667.0 and parameters: {'learning_rate': 0.0747774484504541, 'num_leaves': 300, 'max_depth': 9, 'min_child_samples': 182, 'subsample': 0.9846459514304582, 'colsample_bytree': 0.8929942723073518}. Best is trial 0 with value: 152246.0.
[I 2025-06-17 16:16:46,904] Trial 2 finished with value: 152154.0 and parameters: {'learning_rate': 0.02520389388297912, 'num_leaves': 140, 'max_depth': 10, 'min_child_samples': 125, 'subsample': 0.7559407223181028, 'colsample_bytree': 0.7374614193982663}. Best is trial 2 with value: 152154.0.
[I 2025-06-17 16:21:18,812] Trial 3 finished with value: 153930.0 and parameters: {'learning_rate': 0.071939669306931


Optimisation terminée ! Logging des meilleurs résultats...
Run de résumé des meilleurs résultats créée avec succès dans la run parente.

--- Processus terminé ---


In [8]:
# --- SECTION 1 : CHARGEMENT ET PRÉPARATION DES DONNÉES ---
print("--- 1. Chargement et Préparation des Données ---")
train_df = pd.read_csv('./data/application_train_rdy.csv')
test_df = pd.read_csv('./data/application_test_rdy.csv')
print("Chargement des données train/test réussi.")

if not train_df.empty:
    # Appliquer les mêmes étapes de nettoyage aux deux jeux de données
    for df in [train_df, test_df]:
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        cols_to_check = [col for col in df.columns if col not in ['TARGET', 'SK_ID_CURR']]
        object_cols = df[cols_to_check].select_dtypes(include='object').columns
        if len(object_cols) > 0:
            for col in object_cols:
                df[col] = pd.to_numeric(df[col], errors='coerce')

    # Préparation finale de X, y et X_test
    TARGET = 'TARGET'
    y = train_df[TARGET]
    
    train_ids = train_df['SK_ID_CURR']
    test_ids = test_df['SK_ID_CURR']
    
    # On enlève la cible et l'ID du jeu d'entraînement
    X = train_df.drop(columns=[TARGET, 'SK_ID_CURR'])
    
    # On enlève les mêmes colonnes du jeu de test
    cols_to_drop_from_test = ['SK_ID_CURR']
    if TARGET in test_df.columns:
        cols_to_drop_from_test.append(TARGET)
    X_test = test_df.drop(columns=cols_to_drop_from_test)

    # Nettoyage des noms de colonnes
    X.columns = [re.sub(r'[^A-Za-z0-9_]+', '_', col) for col in X.columns]
    X_test.columns = [re.sub(r'[^A-Za-z0-9_]+', '_', col) for col in X_test.columns]
    
    print(f"Données prêtes avec {X.shape[1]} features.")

    # --- SECTION 2 : ENTRAÎNEMENT DU MODÈLE FINAL ---
    print("\n--- 2. Entraînement du Modèle Final ---")

    # !! IMPORTANT !!
    # Remplacez ces hyperparamètres par ceux trouvés par votre étude Optuna
    best_params = {
        'objective': 'binary',
        'metric': 'auc',
        'verbose': -1,
        'n_jobs': -1,
        'seed': 42,
        'boosting_type': 'gbdt',
        'n_estimators': 2000, # Garder un nombre élevé pour l'early stopping
        # Collez ici les meilleurs paramètres d'Optuna
        'learning_rate': 0.010241101044512771, # Exemple, à remplacer
        'num_leaves': 204,      # Exemple, à remplacer
        'max_depth': 12,        # Exemple, à remplacer
        'min_child_samples': 200,# Exemple, à remplacer
        'subsample': 0.8843947846459445,      # Exemple, à remplacer
        'colsample_bytree': 0.7174115065647507, # Exemple, à remplacer
    }

    # Création du pipeline final
    final_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('classifier', lgb.LGBMClassifier(**best_params))
    ])
    
    X_train_part, X_val_part, y_train_part, y_val_part = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
    
    final_pipeline.fit(
        X_train_part, 
        y_train_part,
        classifier__callbacks=[lgb.early_stopping(100, verbose=False)],
        classifier__eval_set=[(X_val_part, y_val_part)]
    )
    print("Modèle final entraîné.")

    # --- SECTION 3 : ANALYSE DE L'IMPORTANCE DES FEATURES ---
    print("\n--- 3. Analyse de l'Importance des Features ---")
    EXPERIMENT_NAME = "Credit Scoring - Final Model Analysis"
    mlflow.set_experiment(EXPERIMENT_NAME)

    with mlflow.start_run(run_name="Final_LGBM_Optimized"):
        mlflow.log_params(best_params)

        # 3.1 Importance Globale
        print("  - Calcul de l'importance globale des features...")
        feature_importances = final_pipeline.named_steps['classifier'].feature_importances_
        feature_names = X.columns
        importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importances}).sort_values('importance', ascending=False)
        
        plt.figure(figsize=(10, 8))
        sns.barplot(x='importance', y='feature', data=importance_df.head(20))
        plt.title('Top 20 des Features les plus importantes (Global)')
        plt.tight_layout()
        global_importance_path = "global_feature_importance.png"
        plt.savefig(global_importance_path)
        plt.close()
        mlflow.log_artifact(global_importance_path)
        print("  - Graphique de l'importance globale loggué comme artefact.")

        # 3.2 Importance Locale avec SHAP
        print("  - Calcul de l'importance locale (SHAP)...")
        # SHAP a besoin des données imputées
        X_test_imputed = final_pipeline.named_steps['imputer'].transform(X_test)
        
        explainer = shap.TreeExplainer(final_pipeline.named_steps['classifier'])
        shap_values = explainer.shap_values(X_test_imputed)

        is_list_of_arrays = isinstance(shap_values, list)
        
        shap_values_for_plot = shap_values[1] if is_list_of_arrays else shap_values
        expected_value_for_plot = explainer.expected_value[1] if is_list_of_arrays else explainer.expected_value

        # Graphique résumé SHAP (beeswarm)
        shap.summary_plot(shap_values_for_plot, X_test, show=False)
        plt.title("Résumé SHAP de l'impact des features")
        plt.tight_layout()
        shap_summary_path = "shap_summary_plot.png"
        plt.savefig(shap_summary_path)
        plt.close()
        mlflow.log_artifact(shap_summary_path)
        print("  - Graphique résumé SHAP loggué comme artefact.")
        
        # MODIFIÉ : Remplacement du force_plot par des waterfall_plots pour 3 clients aléatoires
        print("  - Création des graphiques waterfall SHAP pour 3 clients aléatoires...")
        
        # Création de l'objet Explanation pour faciliter la manipulation
        explanation = shap.Explanation(
            values=shap_values_for_plot,
            base_values=expected_value_for_plot,
            data=X_test.values,
            feature_names=X_test.columns
        )

        # Sélectionner 3 indices aléatoires
        random_indices = X_test.sample(n=3, random_state=42).index

        for idx in random_indices:
            # Créer le waterfall plot pour une seule observation
            shap.waterfall_plot(explanation[idx], max_display=15, show=False)
            
            client_id = test_ids[idx]
            plt.title(f'SHAP Waterfall Plot pour le client {client_id}')
            plt.tight_layout()
            waterfall_path = f"shap_waterfall_plot_client_{client_id}.png"
            plt.savefig(waterfall_path)
            plt.close()
            
            # Logger l'artefact dans MLflow
            mlflow.log_artifact(waterfall_path)
            print(f"    - Waterfall plot pour le client {client_id} loggué comme artefact.")
        
        # --- SECTION 4 : ENREGISTREMENT ET SERVING DU MODÈLE ---
        print("\n--- 4. Enregistrement du Modèle dans le Model Registry ---")
        
        registered_model_name = "CreditScoringModel"
        
        mlflow.sklearn.log_model(
            sk_model=final_pipeline,
            artifact_path="model",
            registered_model_name=registered_model_name
        )
        print(f"Modèle enregistré dans le Model Registry sous le nom : '{registered_model_name}'")

        # --- Test du "Serving" ---
        print("\n--- Test du Serving en chargeant le modèle depuis le registre ---")
        
        try:
            loaded_model = mlflow.pyfunc.load_model(f"models:/{registered_model_name}/latest")
            print("Modèle chargé avec succès depuis le registre.")

            # On prend un des clients déjà expliqués
            sample = X_test.loc[[random_indices[0]]]
            prediction = loaded_model.predict(sample)
            print(f"\nPrédiction test pour le client {test_ids.loc[random_indices[0]]}:")
            if isinstance(prediction, np.ndarray) and prediction.ndim == 2:
                 print(f"  Probabilité de défaut (classe 1) : {prediction[0][1]:.4f}")
            else:
                 print(f"  Probabilité (classe 1) : {prediction[0]:.4f}")

        except Exception as e:
            print(f"ERREUR lors du chargement ou de la prédiction avec le modèle du registre : {e}")

    print("\n--- Processus Final Terminé ---")

--- 1. Chargement et Préparation des Données ---
Chargement des données train/test réussi.
Données prêtes avec 796 features.

--- 2. Entraînement du Modèle Final ---
Modèle final entraîné.

--- 3. Analyse de l'Importance des Features ---
  - Calcul de l'importance globale des features...
  - Graphique de l'importance globale loggué comme artefact.
  - Calcul de l'importance locale (SHAP)...
  - Graphique résumé SHAP loggué comme artefact.
  - Création des graphiques waterfall SHAP pour 3 clients aléatoires...
    - Waterfall plot pour le client 208550 loggué comme artefact.
    - Waterfall plot pour le client 173779 loggué comme artefact.
    - Waterfall plot pour le client 365820 loggué comme artefact.

--- 4. Enregistrement du Modèle dans le Model Registry ---


2025/06/23 09:45:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modèle enregistré dans le Model Registry sous le nom : 'CreditScoringModel'

--- Test du Serving en chargeant le modèle depuis le registre ---
Modèle chargé avec succès depuis le registre.

Prédiction test pour le client 208550:
  Probabilité (classe 1) : 0.0000

--- Processus Final Terminé ---


Registered model 'CreditScoringModel' already exists. Creating a new version of this model...
Created version '2' of model 'CreditScoringModel'.
